
# Exemplo Básico — Relação N×N (Aluno ↔ Disciplina) com SQLAlchemy

Objetivo: demonstrar um relacionamento muitos-para-muitos entre as tabelas **Aluno** e **Disciplina**, com operações simples de **adicionar matrícula** e **remover matrícula**.


## Setup: Engine, Base, Modelos e Relacionamento N×N

In [7]:

from sqlalchemy import create_engine, Column, Integer, String, Table, ForeignKey
from sqlalchemy.orm import declarative_base, relationship, sessionmaker

DB_PATH = "../data/escola.db"
engine = create_engine(f"sqlite:///{DB_PATH}", echo=False)
Base = declarative_base()

# Tabela de associação (N×N) Aluno-Disciplina
aluno_disciplina = Table(
    "aluno_disciplina",
    Base.metadata,
    Column("aluno_id", Integer, ForeignKey("alunos.id"), primary_key=True),
    Column("disciplina_id", Integer, ForeignKey("disciplinas.id"), primary_key=True),
)

class Aluno(Base):
    __tablename__ = "alunos"
    id = Column(Integer, primary_key=True, autoincrement=True)
    nome = Column(String, nullable=False)

    # Muitos-para-muitos via tabela secundária
    disciplinas = relationship(
        "Disciplina",
        secondary=aluno_disciplina,
        back_populates="alunos",
    )

    def __repr__(self):
        return f"Aluno(id={self.id}, nome={self.nome!r})"

class Disciplina(Base):
    __tablename__ = "disciplinas"
    id = Column(Integer, primary_key=True, autoincrement=True)
    nome = Column(String, nullable=False, unique=True)

    alunos = relationship(
        "Aluno",
        secondary=aluno_disciplina,
        back_populates="disciplinas",
    )

    def __repr__(self):
        return f"Disciplina(id={self.id}, nome={self.nome!r})"

Base.metadata.drop_all(engine)   # limpar caso reexecute
Base.metadata.create_all(engine)

Session = sessionmaker(bind=engine)
session = Session()

print("Banco criado em:", DB_PATH)


Banco criado em: ../data/escola.db


## Inserir dados básicos

In [8]:

# Alunos
a1 = Aluno(nome="Carlos")
a2 = Aluno(nome="Ana")
a3 = Aluno(nome="Mariana")

# Disciplinas
d1 = Disciplina(nome="Matemática")
d2 = Disciplina(nome="Programação")
d3 = Disciplina(nome="História")

session.add_all([a1, a2, a3, d1, d2, d3])
session.commit()

print("Alunos:", session.query(Aluno).all())
print("Disciplinas:", session.query(Disciplina).all())


Alunos: [Aluno(id=1, nome='Carlos'), Aluno(id=2, nome='Ana'), Aluno(id=3, nome='Mariana')]
Disciplinas: [Disciplina(id=1, nome='Matemática'), Disciplina(id=2, nome='Programação'), Disciplina(id=3, nome='História')]


## Matricular (adicionar relacionamentos)

In [9]:

# Carlos em Matemática e Programação
a1.disciplinas.extend([d1, d2])

# Ana em Programação
a2.disciplinas.append(d2)

# Mariana em História
a3.disciplinas.append(d3)

session.commit()

# Listar matrículas: Aluno -> Disciplinas
for aluno in session.query(Aluno).all():
    nomes = [disc.nome for disc in aluno.disciplinas]
    print(f"{aluno.nome}: {nomes}")


Carlos: ['Matemática', 'Programação']
Ana: ['Programação']
Mariana: ['História']


## Consultar pelo lado da Disciplina

In [5]:

# Listar: Disciplina -> Alunos
for disc in session.query(Disciplina).all():
    nomes = [aluno.nome for aluno in disc.alunos]
    print(f"{disc.nome}: {nomes}")


Matemática: ['Carlos']
Programação: ['Carlos', 'Ana']
História: ['Mariana']


## Desmatricular (remover relacionamentos)

In [6]:

# Remover Carlos de Programação (remove linha da tabela de associação)
a1.disciplinas.remove(d2)
session.commit()

# Remover Ana de TODAS as disciplinas
a2.disciplinas.clear()
session.commit()

# Conferir após remoções
print("Após remoções:")
for aluno in session.query(Aluno).all():
    nomes = [disc.nome for disc in aluno.disciplinas]
    print(f"{aluno.nome}: {nomes}")


Após remoções:
Carlos: ['Matemática']
Ana: []
Mariana: ['História']



### Observações rápidas
- O relacionamento N×N é modelado via uma **tabela de associação** (`aluno_disciplina`), sem classe própria.  
- `append`, `extend`, `remove` e `clear` manipulam as linhas na tabela de associação.  
- A exclusão de um **Aluno** ou **Disciplina** não está demonstrada aqui; o foco é adicionar/remover **relacionamentos**.
